In [1]:
import pandas as pd 
from datetime import datetime   

In [26]:
url='/Users/sonicboy66/Documents/Code_Practice/women_health_trials.csv'
df= pd.read_csv(url)


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   trial_id              7 non-null      object
 1   drug_name             6 non-null      object
 2   dose_mg               5 non-null      object
 3   start_date            7 non-null      object
 4   condition             7 non-null      object
 5   intervention_details  7 non-null      object
dtypes: object(6)
memory usage: 464.0+ bytes


In [28]:
df['drug_name'].unique()

array(['Yaz', 'Lupron', 'Estrace', 'Mirena', 'Letrozole', 'Clomid', nan],
      dtype=object)

In [29]:
# creating a map for drug brand to generic name conversion. also drug map is case sensitive

drug_map = { ## this is grok version 
    'yaz': 'Drospirenone and Ethinyl Estradiol',
    'lupron': 'Leuprolide',
    'estrace': 'Estradiol',
    'mirena': 'Levonorgestrel',
    'letrozole': 'Letrozole',
    'clomid': 'Clomiphene Citrate'
}

In [30]:
# create a function to normalize the dates 
def normalize_date(date_str):
    try:
        if pd.isnull(date_str):
            return None
        return datetime.strptime(date_str, '%Y-%m-%d').date()
    except ValueError:
        return None

In [31]:
# create a function that normalize the drug name and check NA
def normalize_drug(drug_name):
    try:
        if pd.isnull(drug_name):
            return None
    except TypeError:
        return None
    return drug_map.get(drug_name.strip().lower(), drug_name.strip().lower())    

In [32]:
df["drug_generic"] = df["drug_name"].apply(normalize_drug)


In [34]:
# normalized dates
df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')

In [44]:
# Extract unit part (optional letters after number, e.g. "mg")
df['dose_unit'] = df['dose_mg'].str.extract(r'([a-zA-Z]+)').fillna('None')

# Extract numeric part (handles numbers with optional space before units)
df['dose'] = df['dose_mg'].str.extract(r'(\d*\.?\d+)').astype(float)


In [46]:
df_final = df.drop(columns=['dose_mg'])

In [49]:
df_final

,trial_id,drug_name,start_date,condition,intervention_details,drug_generic,dose_unit,dose
0,WH001,Yaz,2022-10-01,polycystic ovary syndrome,Drospirenone/ethinyl estradiol daily,Drospirenone and Ethinyl Estradiol,mg,3.00
1,WH002,Lupron,2022-11-15,Endometriosis,3.75mg Lupron Depot injection,Leuprolide,None,3.75
2,WH003,Estrace,2023-01-02,Menopause symptoms,Estradiol oral tablet once daily,Estradiol,mg,1.00
3,WH004,Mirena,2023-02-15,contraception,Levonorgestrel-releasing IUD,Levonorgestrel,None,NaN
4,WH005,Letrozole,2023-01-05,infertility,2.5 mg Letrozole daily,Letrozole,None,2.50
5,WH006,Clomid,2023-01-07,Ovulatory dysfunction,Clomiphene Citrate oral tablet,Clomiphene Citrate,mg,50.00
6,WH007,NaN,2023-01-08,PMS,"Vitamin B6, magnesium",None,None,NaN
